In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max.columns',500) # Unuk menampilkan semua kolom


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd './drive/MyDrive/Hacktiv8/FINAL PROJECT/F1'

[Errno 2] No such file or directory: './drive/MyDrive/Hacktiv8/FINAL PROJECT/F1'
/content/drive/MyDrive/Hacktiv8/FINAL PROJECT/F1


In [ ]:
model = pd.read_csv("./rideshare_model.csv")
model.head()

,cab_type,name,price,distance,surge_multiplier
0,Lyft,Shared,5.0,0.44,1.0
1,Lyft,Lux,11.0,0.44,1.0
2,Lyft,Lyft,7.0,0.44,1.0
3,Lyft,Lux Black XL,26.0,0.44,1.0
4,Lyft,Lyft XL,9.0,0.44,1.0


# **Label encoder**

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoder(data, feature):
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature])
    return data

new_df_model = model.copy()

for col in new_df_model.select_dtypes(include='O').columns:
    new_df_model = label_encoder(new_df_model, col)

new_df_model.head()


,cab_type,name,price,distance,surge_multiplier
0,0,7,5.0,0.44,1.0
1,0,2,11.0,0.44,1.0
2,0,5,7.0,0.44,1.0
3,0,4,26.0,0.44,1.0
4,0,6,9.0,0.44,1.0


In [ ]:
from sklearn.model_selection import train_test_split
X = new_df_model.drop(columns=['price'],axis=1).values
y = new_df_model['price'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()

In [ ]:
# Fit to data training
linear_reg.fit(X_train,y_train)
y_pred=linear_reg.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.4982111627704644

Dapat dilihat bahwa nilai r^2 terbilang cukup rendah

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(mse)
print(rmse)

38.70865656429561
6.221628128094414


MSE dikatakan memiliki kinerja baik jika nilainya semakin kecil. Tetapi nilai RMSE disini cukup baik, karena nilainya yang kecil,, yang artinya kesalahan rata-rata dalam prediksi dalam unit yang sama dengan variabel dependen kecil.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

def find_best_model_using_gridsearchcv(X, y):
    model = LinearRegression()

    # Parameter yang akan diuji
    param_grid = {
        'fit_intercept': [True, False],
        'positive': [True, False],
        'copy_X': [True, False],
        'n_jobs': [None, 1, 2, 3, 4]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f'Best Parameters: {best_params}')
    print(f'Best R^2 Score: {best_score}')

# Panggil fungsi dengan X dan y yang sesuai
find_best_model_using_gridsearchcv(X, y)


Best Parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}
Best R^2 Score: 0.4990716002644257


Meskipun telah dilakukan pengaturan parameter untuk memperoleh r^2 score yang lebih baik dengan GridSearchCV, nilai yang didapatkan tidak jauh berbeda.

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, ShuffleSplit
import pandas as pd

def find_best_model_using_gridsearchcv(X, y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False],
                'positive': [True, False],
                'copy_X': [True, False],
                'n_jobs': [None, 1, 2, 3, 4]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X, y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

import warnings
warnings.filterwarnings('ignore')

# Panggil fungsi dengan X dan y yang sesuai
find_best_model_using_gridsearchcv(X, y)


,model,best_score,best_params
0,linear_regression,0.498746,"{'copy_X': True, 'fit_intercept': True, 'n_job..."
1,lasso,0.461509,"{'alpha': 1, 'selection': 'cyclic'}"
2,decision_tree,0.964413,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


# **Kesimpulan**

Dapat disimpulkan bahwa proses encoding menggunakan Label Encode tidak cocok, karena memberikan nilai R^2 yang rendah. dan dari sini dapat diperoleh kesimpulan lain dimana model decision tree sangat cocok untuk data ini.

Kembali ke [Uber and Lyft Boston](https://colab.research.google.com/drive/1DlBEYAfh0Qiq_qtPDX0AMG4TrD7ujzyp?usp=sharing)